In [1]:
import argparse
import qlib
import ruamel.yaml as yaml
from qlib.utils import init_instance_by_config
import pandas as pd
import torch

In [2]:
parser = argparse.ArgumentParser(allow_abbrev=False)
parser.add_argument("--seed", type=int, default=1000, help="random seed")
parser.add_argument("--config_file", type=str, default="us360_nas.yaml", help="config file")
parser.add_argument("-f")
args = parser.parse_args()


seed = args.seed
config_file = args.config_file

# set random seed
with open(config_file) as f:
    config = yaml.safe_load(f)

# seed_suffix = "/seed1000" if "init" in config_file else f"/seed{seed}"
seed_suffix = ""
config["task"]["model"]["kwargs"].update(
    {"seed": seed, "logdir": config["task"]["model"]["kwargs"]["logdir"] + seed_suffix}
)

# initialize workflow
qlib.init(
    provider_uri=config["qlib_init"]["provider_uri"],
    region=config["qlib_init"]["region"],
)

[818:MainThread](2021-10-22 00:43:15,176) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[818:MainThread](2021-10-22 00:43:15,179) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[818:MainThread](2021-10-22 00:43:15,369) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[818:MainThread](2021-10-22 00:43:15,370) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': PosixPath('/home/us_data_dump')}


In [3]:
latest_dataset = init_instance_by_config(config["task"]["dataset"])

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
[818:MainThread](2021-10-22 00:46:04,670) INFO - qlib.timer - [log.py:113] - Time cost: 169.283s | Loading data Done
/usr/local/lib/python3.8/dist-packages/numpy/lib/nanfunctions.py:997: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
[818:MainThread](2021-10-22 00:46:05,262) INFO - qlib.timer - [log.py:113] - Time cost: 0.587s | RobustZScoreNorm Done
[818:MainThread](2021-10-22 00:46:05,354) INFO - qlib.timer - [log.py:113] - Time cost: 0.090s | Fillna Done
[818:MainThread](2021-10-22 00:46:05,373) INFO - qlib.timer - [log.py:113] - Time cost: 0.015s | CSRankNorm D

In [4]:
model = init_instance_by_config(config["task"]["model"])

Please install necessary libs for CatBoostModel.
Please install necessary libs for XGBModel, such as xgboost.


[818:MainThread](2021-10-22 00:46:06,102) WARNING - qlib.TRA - [pytorch_tra.py:100] - `eval_train` will be ignored when using TRA.router
[818:MainThread](2021-10-22 00:46:06,109) WARNING - qlib.TRA - [pytorch_tra.py:132] - logdir output/basic_nasdaq is not empty
[818:MainThread](2021-10-22 00:46:06,110) INFO - qlib.TRA - [pytorch_tra.py:141] - init TRAModel...
[818:MainThread](2021-10-22 00:46:06,119) INFO - qlib.TRA - [pytorch_tra.py:171] - # model params: 55936
[818:MainThread](2021-10-22 00:46:06,120) INFO - qlib.TRA - [pytorch_tra.py:172] - # tra params: 5606


RNN(
  (rnn): LSTM(6, 64, num_layers=2, batch_first=True)
  (W): Linear(in_features=64, out_features=64, bias=True)
  (u): Linear(in_features=64, out_features=1, bias=False)
)
TRA(
  (predictors): Linear(in_features=128, out_features=3, bias=True)
  (router): LSTM(3, 32, batch_first=True)
  (fc): Linear(in_features=160, out_features=3, bias=True)
)


In [5]:
state_dict = torch.load("/home/qlib_test/examples/benchmarks/TRA/output/basic_nasdaq/model.bin", map_location="cpu")["model"]

In [6]:
model.model.load_state_dict(state_dict)

<All keys matched successfully>

In [7]:
model.fitted = True

In [8]:
pred = model.predict(latest_dataset)

100%|████████████████████████████████████████████████████████| 14/14 [00:01<00:00,  7.52it/s]
[818:MainThread](2021-10-22 00:46:08,036) INFO - qlib.TRA - [pytorch_tra.py:516] - test metrics: {'MSE': 0.1672586483174663, 'MAE': 0.3318317780500833, 'IC': -0.0035558088367162962, 'ICIR': -0.04146635819306294}


In [9]:
latest_dataset.to_pickle(path="/home/us_dataset_basic.pkl")

In [10]:
# pred

In [11]:
reset_df = pred.reset_index('instrument')
# reset_df

In [12]:
from IPython.display import clear_output
import numpy as np
indexs = reset_df.index.drop_duplicates()
cnt = 0 

for idx in indexs:
    # cnt+=1
    # if cnt < 800: continue
    print(idx)
    if type(reset_df.loc[idx]['score']) is np.float32:
        stack = reset_df.loc[idx]
    else:
        stack = reset_df.loc[idx].sort_values(by='score' , ascending=False)
    print(stack)
    stack.to_csv('/home/results_basic/'+str(idx)+'.csv', sep=',', na_rep='NaN')
    # input()
    # clear_output()

2021-10-01 00:00:00
           instrument     score     label   score_0   score_1   score_2
datetime                                                               
2021-10-01      LBTYK  0.200630  1.642660  0.200630 -0.043826  0.045602
2021-10-01        CAG  0.199698 -0.124291  0.199698 -0.019648  0.058338
2021-10-01         IR  0.197308  0.131010  0.197308 -0.016129  0.044707
2021-10-01       NTES  0.196862  0.540835  0.196862 -0.021479  0.046701
2021-10-01        PEG  0.196644 -1.199243  0.196644 -0.028511  0.059139
...               ...       ...       ...       ...       ...       ...
2021-10-01       NXPI -0.056637  0.681922  0.168374 -0.056637  0.065682
2021-10-01       ILMN -0.057095  1.199243  0.163615 -0.057095  0.056225
2021-10-01         JD -0.057848  1.609068  0.176583 -0.057848  0.046729
2021-10-01       MRNA -0.064789  1.071592  0.163381 -0.064789  0.060057
2021-10-01       EXPD -0.066587 -0.104136  0.179157 -0.066587  0.059860

[515 rows x 6 columns]
2021-10-04 00:00:00


In [13]:
reset_df.to_pickle(path="/home/reset_df_basic.pkl")